<a href="https://colab.research.google.com/github/toussaintma/neuralnetworksfromzerotohero/blob/main/walkthrough_makemore_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
# data and code at https://github.com/karpathy/makemore
# course at https://www.youtube.com/watch?v=PaCmpygFfXo&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=2&t=11s

In [52]:
import pandas as pd

In [53]:
df_words = pd.read_csv('names.txt', header=None, names=['name'])

In [54]:
df_words.shape

(32033, 1)

In [55]:
df_words['name']

0            emma
1          olivia
2             ava
3        isabella
4          sophia
           ...   
32028       zylas
32029       zyran
32030       zyrie
32031       zyron
32032       zzyzx
Name: name, Length: 32033, dtype: object

In [56]:
stat = {}
for w in df_words.loc[:, 'name']:
  for ch1, ch2 in zip(w, w[1:]):
    #print(ch1, ch2, sep='')
    if ch1 + ch2 in stat:
      stat[ch1 + ch2] += 1
    else:
      stat[ch1 + ch2] = 1

In [57]:
def get_index(c):
  return ord(c) - ord('a')
get_index('e')

4

In [59]:
stats = [[0 for i in range(26)] for j in range(26)]
len(stats)

26

In [61]:
for k, v in stat:
  print(k)
  stats[get_index(k[0])][get_index(k[1])] = v
stats

e


IndexError: ignored